In [117]:
base_path = "/Users/lorenapacheco/Concordia/Masters/"
file_analysis_folder= base_path + "BugReportsMining/coverageMining/"
coverage_data_folder = base_path + "coverage_data/"
out = '/Users/lorenapacheco/Concordia/Masters/BugReportsMining/ochiaiScores/'

buggy_commits_info = {
    "fastjson": {
        #"2306": "0c45401c938ef866700e602adda16a4a0f55960e",
        "2351": "8e985930057c21c95a3065ca9f2cad3b5e42d4ba",
        "3119": "307f6637b9f8502a081468443c2b3c85cea5e545",
        "3280": "cc7dfab3dde888b81577fb18c0eb223761b5e31f",
        #"3309": "7ffa2a013be4401df24455c2cefa2a52af5eebbe",
        "3637": "00cdc53606111e7802f8f5559d44feeccba2657f",
    },
    "commons-csv": {
        "CSV-100": "2b5f84ede12cfadd7946ffb07a56709b8322a02f"
    },
    "jsoup": {
        "1098": "fb50d9635874185fc050735057b8a7363cd53f3f",
        "1218": "8d1d503913a68e549b5c4a94717c62cf3f64507a",
        "1251": "de97030ff54ee0bd306cbc58bd8093645cc8a5dc",
        "1274": "de1ced99d4dae991a546534454d5f7d9dc26f0b1",
        "1324": "528ba552b19ab2ae949feecb373ef85a0b126566",
        "740": "222feb1791388d1e94e2c99bb5858cf160904d16",
        "968": "d32321351ced83c7c2edff2abe4a8b2fa17a6bd7",
        "980": "71561e09e0f29ec5fd1bb918206f3d8e42876518",
        "990":  "02668f757c59f0c1a7ad8f3169faf061b4b787c1"
    },
    "junit4": {
        "1178": "c07965ab1b232b04a13ea84fdbcdd8be04ea0182"
    }
}

In [120]:
import glob
import os
import json
import math

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def dict_to_json_file(file, dic, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

for bug_report_analysis_file in glob.glob(file_analysis_folder + "*/*.json"):
    project = bug_report_analysis_file.split("/")[-2]
    bug_id = bug_report_analysis_file.split("/")[-1].replace(".json","")
    print(project + " ---- " + bug_id)
    analysis_data = json_file_to_dict(bug_report_analysis_file)
    buggy_commit = buggy_commits_info [project][bug_id]
    coverage_details_folder = coverage_data_folder + project + "/" + buggy_commit + "/details/"
    fake_failed_tests = []
    fake_tests_status = []
    methods_list = []


    print("* Part 1")
    for st_entry in analysis_data.keys():
        if "unique_tests_that_cover_the_method" in analysis_data[st_entry] and analysis_data[st_entry]["unique_tests_that_cover_the_method"] != []:
            fake_failed_tests += analysis_data[st_entry]["unique_tests_that_cover_the_method"]
    if not fake_failed_tests:
        for st_entry in analysis_data.keys():
            if analysis_data[st_entry]["tests_that_cover_the_method"] != "N/A":
                fake_failed_tests += analysis_data[st_entry]["tests_that_cover_the_method"]
    fake_failed_tests = list(set(fake_failed_tests))

    print("* Part 2")
    for test_coverage_file in glob.glob(coverage_details_folder + "*.json"):
        test_file_name = test_coverage_file.split("/")[-1]
        if test_file_name in fake_failed_tests:
            fake_tests_status.append((test_file_name, 1)) # error flag
        else:
            if "coverage_details" not in test_file_name:
                fake_tests_status.append((test_file_name, 0)) # no error flag
        coverage_data = json_file_to_dict(test_coverage_file)
        if "coverage_details" not in test_file_name:
            for method_id in coverage_data["covered_stats"].keys():
                method_name = method_id
                if "(" in method_name:
                    method_name = method_name.split("(")[0]
                else:
                    method_name = method_name.split(" {")[0]
                try:
                    method_name = method_name.split(".")[-2] + "." + method_name.split(".")[-1]
                except:
                    print("#####")
                    print(method_name)
                    print(method_id)
                if method_name not in methods_list:
                    methods_list.append(method_name)

    print("* Part 3")
    # Pre-populating the matrix
    methods_execution_matrix = []
    for i in range(0, len(fake_tests_status)):
        pre_population_list_auxiliar=[]
        for j in range(0, len(methods_list)):
            pre_population_list_auxiliar.append(0)
        methods_execution_matrix.append(pre_population_list_auxiliar)


    for test_pos in range(0, len(fake_tests_status)):
        test = fake_tests_status[test_pos]
        test_name = test[0].replace("[", "*").replace("]", "*")
        test_coverage_file = glob.glob(coverage_details_folder + test_name)[0]
        test_coverage_data = json_file_to_dict(test_coverage_file)
        for method_pos in range(0, len(methods_list)):
            method = methods_list[method_pos]
            for method_covered_by_the_test in test_coverage_data["covered_stats"].keys():
                if "." + method + "("  in method_covered_by_the_test or "." + method + " {" in method_covered_by_the_test:
                    methods_execution_matrix[test_pos][method_pos] = 1
                    break


    print("* Part 4")
    methods_ochiai_scores = {}
    for method_pos2 in range(0, len(methods_list)):
        n00 = 0
        n01 = 0
        n10 = 0
        n11 = 0
        s_o = 0
        method_name = methods_list[method_pos2]
        for test_pos2 in range(0, len(fake_tests_status)):
            if methods_execution_matrix[test_pos2][method_pos2] == 1:
                if fake_tests_status[test_pos2][1] == 1:
                    n11 += 1
                else:
                    n10 += 1
            else:
                if fake_tests_status[test_pos2][1] == 1:
                    n01 += 1
                else:
                    n00 += 1
        try:
            s_o = n11/math.sqrt((n11+n01)*(n11+n10))
        except ZeroDivisionError:
            s_o = 0
        methods_ochiai_scores[method_name] = s_o

    print("Number of fake passing tests: " + str(len(fake_tests_status)-len(fake_failed_tests)))
    print("Number of fake failing tests: " + str(len(fake_failed_tests)) + "\n")
    dict_to_json_file(bug_id, methods_ochiai_scores, out + project + "/")

print("Execution completed")




fastjson ---- 3280
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 1787
Number of fake failing tests: 436

fastjson ---- 2351
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 676
Number of fake failing tests: 1420

fastjson ---- 3637
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 1074
Number of fake failing tests: 1213

fastjson ---- 3119
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 1520
Number of fake failing tests: 679

commons-csv ---- CSV-100
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 180
Number of fake failing tests: 2

jsoup ---- 990
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 10
Number of fake failing tests: 22

jsoup ---- 968
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 7
Number of fake failing tests: 24

jsoup ---- 980
* Part 1
* Part 2
* Part 3
* Part 4
Number of fake passing tests: 4
Number of fake failing tests: 28

jsoup ---- 1274
* P